**GRUPO 7**

Beatriz Herguedas Pinedo

Pablo Hernández Aguado

In [38]:
from search import *
from random import randint

In [11]:
class ProblemaGenetico(object):
    def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness,longitud_individuos):
        self.genes = genes
        self.fun_dec = fun_dec
        self.fun_cruza = fun_cruza
        self.fun_muta = fun_muta
        self.fun_fitness = fun_fitness
        self.longitud_individuos = longitud_individuos
        """Constructor de la clase"""

    def decodifica(self, genotipo):
        """Devuelve el fenotipo a partir del genotipo"""
        fenotipo = self.fun_dec(genotipo)
        return fenotipo
    def muta(self, cromosoma,prob):
        """Devuelve el cromosoma mutado"""   
        mutante = self.fun_muta(cromosoma,prob)
        return mutante
    def cruza(self, cromosoma1, cromosoma2):         
        """Devuelve el cruce de un par de cromosomas"""
        cruce = self.fun_cruza(cromosoma1,cromosoma2)
        return cruce 
    def fitness(self, cromosoma):    
        """Función de valoración"""
        valoracion = self.fun_fitness(cromosoma)
        return valoracion

## Problema de Asignación de Tareas.

En una empresa hay `n` tareas que realizar, `t1, ..., tn`, y `m` trabajadores `T1, ..., Tm` con distintos niveles de cualificación. Es decir, un trabajador `Ti` puede ser capaz de resolver varias de las tareas de la empresa. Los trabajadores no pueden realizar varias tareas simultáneamente y tardarán un tiempo en realizarla. Si un trabajador `Ti` puede realizar la tarea `tj`, entonces denotaremos por `dij` el tiempo que el trabajador `i` tarda en realizar la tarea `j`.

Existen restricciones temporales ya que algunas tareas no pueden comenzar hasta que otras no hayan terminado. Esto es, tenemos un conjunto de restricciones temporales `ti < tj` que indican que `ti` se debe realizar antes de `tj`. Por ejemplo:

`generar informe ventas < imprimir informe ventas`

Se pide dar una solución al problema de decidir, para cada tarea, qué trabajador la realiza y en qué momento debe comenzar a realizar la tarea para que, al final, todas las tareas estén realizadas, todas las restricciones temporales se satisfagan y el conjunto de las tareas se realice en el menor tiempo posible.

## Propuesta de resolución.

Para resolver este problema, utilizaremos un algoritmo genético, cuya implementación detallamos a continuación.

#### Representación del problema.

En primer lugar, hemos de definir la forma que tendran los cromosomas que utilizará el algoritmo genético para resolver el problema. 

Dadas `n` tareas, los cromosomas consistirán en una cadena de `n` números, en de forma que si en la posición `j`, el valor asignado es `i`, se deduce que el trabajador `Ti` se encargará de la tarea `tj`. Claramente, por las dependencias entre tareas, puede haber y habrá trabajadores que realicen más de una tarea, por lo que aparecerán varias veces en el _cromosoma_.

Para facilitar la implementación en código del problema, dadas `n` tareas, las enumeraremos comenzando desde 0, de forma que quedan: `t0, t1, ..., tn-1`; ocurriendo lo mismo para los `m` trabajadores.

|  0 |  1 |  2 | ... |  n-1 |
|:--:|:--:|:--:|:---:|:----:|
| i0 | i1 | i2 | ... | in-1 |

Por otro lado, aparece la información inicial del problema (que no los estados iniciales) que nos proporciona información sobre el tiempo de ejecución de una tarea por un trabajador y las interdependencias entre las distintas tareas.

El **tiempo de ejecución de las tareas** estará representado por una matriz D, de dimensión `n x m`, de forma que el valor `dij` representa, como en el enunciado, el tiempo que tarda el trabajador `i` en hacer la tarea `j`. Para aquellos trabajadores que no estén cualificados para hacer alguna tarea, el valor de `dij` será `0`.

También utilizaremos una matriz F, de dimensión `n x n` para guardar las **dependencias entre tareas**, de forma que, dado el elemento `fij`:
- Si este es igual a 0, la tarea `ti` no depende de la tarea `tj`.
- Si este es igual a 1, la tarea `ti` depende de que se haga previamente la tarea `tj`.

Los elementos simétricos de esta matriz no pueden ser ambos igual a 1, es decir, `dij = dji = 1`, pues esto supondría que ninguna de las dos tareas se podrían realizar, al depender una de la otra. Esto provocaría que nuestra función que comprueba el tiempo entrase en un bucle infinito.

#### Problema del trabajo simultáneo.

El enunciado del problema prohíbe que un trabajador pueda realizar 2 tareas al mismo tiempo, cosa que debemos tener en cuenta a la hora de calcular el tiempo que dedica un trabajador a terminar todas sus tareas asignadas.

Así, el tiempo que tarde cada trabajador en realizar todas sus tareas será una cota inferior del tiempo total. Como las dependencias pueden generar espacios de tiempo en que un trabajador esté desocupado porque necesite que acabe una tarea ajena para comenzar la suya, hemos de encontrar una forma de medir este tiempo. Así, el último trabajador que acabe con sus tareas indica el tiempo que se ha necesitado para hacer todas ellas.

Para facilitar el cálculo de estos tiempos, condicionamos a que las tareas estén ya _ordenadas_ en su enumeración, en el sentido de que si una tarea depende de otra, digamos `ti < tj` entonces se dé `i < j`. De esta forma, podemos recorrer el cromosoma desde el principio, utilizando:
- Un _vector_ auxiliar `TF` que nos indique en qué tiempo se ha terminado la tarea `i` para cada posición del vector.
- Un _vector_ `TT` que indique el tiempo que lleva _gastado_ cada trabajador en hacer sus tareas.

_Aclaración: trabajamos de forma inversa con los costes (es decir, negándolos), de forma que se transforma en un problema de maximización. Así, podemos utilizar ceros en la matriz de tiempos de las tareas en vez de infinitos._

El método es el siguiente: se recorre el cromosoma de izquierda a derecha. Para cada posición (tarea `j`, trabajador `j`): 
- Si la tarea no es dependiente, entonces restamos su tiempo de realización `dij` al tiempo de trabajo del trabajador, `TT[i] -= dij`
- Si la tarea es dependiente de otras, vemos los tiempos de finalización de esas tareas, quedándonos con el menor `Mdep`; de forma que si el tiempo de trabajo acumulado del trabajador que va a realizar la tarea dependiente es mayor  que `Mdep`, este debe actualizarse a `Mdep` antes de restarle el tiempo `dij` de la nueva tarea.
En ambos casos, el tiempo de finalización de la tarea es `TT[i]`, así que actualizamos el vector auxiliar `TF[j] = TT[i]`.

El valor mínimo del vector `TT`, negado, nos da el tiempo que tarda en acabar el último trabajador, esto es, en realizarse todas las tareas.

Nótese que el orden en que aparecen las tareas en el cromosoma puede afectar a los tiempos en que algunos trabajadores tienen que esperar a que se acabe una tarea para hacer la suya. En efecto, si un trabajador tiene varias tareas asignadas y realiza primero las tareas sin dependencias, esto aumentará el tiempo de espera del otro trabajador. 

Sin embargo, no necesitamos una función de decodificación totalmente precisa para conseguir una solución subóptima; no obstante, podríamos aplicar el algoritmo genético a cada permutación (que mantenga el orden de dependencia) para hallar una solución más óptima (e incluso aplicar un algoritmo genético a este proceso, que para cada cromosoma de tareas enumeradas aplique el algoritmo genético que vamos a implementar).

#### Función de selección (fitness).

La implementamos siguiendo el método expuesto en el apartado anterior:

In [53]:
def fitnessAsig(nTrab, mTarea, cromosoma, tiemposMat, depMat):
    tT = [0 for i in range(nTrab)]
    tF = [0 for j in range(mTarea)]
    
    for jTarea in range(mTarea):
        iTrabajador = cromosoma[jTarea]
        
        # Dependencias.
        maxTime = 1 # Trabajamos con negativos.
        for kTarea in range(mTarea):
            dep = depMat[jTarea][kTarea]
            if dep == 1:
                maxTime = max(maxTime, tF[kTarea])
        
        if maxTime != 1:
            tT[iTrabajador] = maxTime
            
        # Tarea a realizar.
        dij = tiemposMat[iTrabajador][jTarea]
        tT[iTrabajador] += dij
        tF[jTarea] = tT[iTrabajador]
    
    maxTotal = max(tT)
    
    return maxTotal        

#### Función de cruce.

Para la función de cruce, haremos un simple cruce sobre un punto, pero tomando este de forma aleatoria para conseguir una mayor diversidad de cromosomas.

In [54]:
def cruceAsig(cromosoma1, cromosoma2):
    l1 = len(cromosoma1)
    l2 = len(cromosoma2)
    pCorte = random.randint(0, l1 - 1)
    cruce1 = cromosoma1[0:pCorte] + cromosoma2[pCorte:l2]
    cruce2 = cromosoma2[0:pCorte] + cromosoma1[pCorte:l1]
    
    return [cruce1, cruce2]

#### Función de mutación.

Para la función de mutación, elegimos una posición al azar del cromosoma y, bajo una probalidad, modificamos en 1 el valor asignado a esa posición; es decir, cambiamos el trabajador asignado a esa tarea por el siguiente en la lista de trabajadores.

In [66]:
def mutacionAsig(nTrab, mTarea, cromosoma, prob):
    p = random.randint(0, mTarea - 1)
    if prob > random.uniform(0, 1):
        cromosoma[p] = (cromosoma[p] + 1) % nTrab
        
    return cromosoma

#### Función de decodificación.

Procedemos en esta función de forma análoga a la función de fitness, pero generando la solución obtenida:
- Una lista de elementos `((i, j), (Ti, Tf))` que indica que el trabajador `Ti` tiene asignada la tarea `tj`, que comienza a realizar en el tiempo `Ti` y acaba en tiempo `Tf`.
- Un valor `T*` que indica el tiempo necesario para realizar todas las tareas.

In [92]:
def decodeAsig(nTrab, mTarea, cromosoma, tiemposMat, depMat):
    tT = [0 for i in range(nTrab)]
    tF = [0 for j in range(mTarea)]
    
    asignaciones = list()
    
    for jTarea in range(mTarea):
        iTrabajador = cromosoma[jTarea]
        
        # Dependencias.
        maxTime = -1 # Trabajamos con negativos.
        for kTarea in range(mTarea):
            dep = depMat[jTarea][kTarea]
            if dep == 1:
                maxTime = max(maxTime, tF[kTarea])
        
        if maxTime > tT[iTrabajador]:
            tT[iTrabajador] = maxTime
            
        ti = tT[iTrabajador]
            
        # Tarea a realizar.
        dij = tiemposMat[iTrabajador][jTarea]
        tT[iTrabajador] += dij
        tF[jTarea] = tT[iTrabajador]
        
        tf = tF[jTarea]
        
        asignaciones.append( ( (iTrabajador, jTarea) , (ti, tf) ) )
    
    maxTotal = max(tT)
    
    return (tuple(asignaciones), maxTotal) 

#### Población inicial.

Utilizamos una función simple (correspondiente al anterior apartado de la práctica) que genera cromosomas de forma aleatoria a partir de los genes disponibles (esto es, la enumeración del número de trabajadores empezando en 0, `(0, ..., m - 1)`).

In [57]:
def poblacion_inicial(problema_genetico, tamaño):
    N, M = tamaño, problema_genetico.longitud_individuos
    genes = problema_genetico.genes
    poblacion = list()
    for i in range(N):
        poblacion.append( [random.choice(genes) for j in range(M)] )
        
    return poblacion

#### Funciones auxiliares del apartado anterior.

In [58]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion, k)
        seleccionado = opt(participantes, key = problema_genetico.fitness)
        seleccionados.append(seleccionado)
        #poblacion.remove(seleccionado)
    return seleccionados  

In [78]:
def algoritmo_genetico(problema_genetico, k, opt, ngen, size, prop_cruces, prob_mutar):
    poblacion = poblacion_inicial(problema_genetico, size)
    n_padres = round(size * prop_cruces)
    n_padres = int (n_padres if n_padres % 2 == 0 else n_padres - 1)
    n_directos = size - n_padres
    assert k <= n_padres
    assert k <= n_directos
    for _ in range(ngen):
        poblacion = nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar)

    mejor_cr = opt(poblacion, key = problema_genetico.fitness)
    mejor = problema_genetico.decodifica(mejor_cr)
    return mejor

In [47]:
def nueva_generacion(problema_genetico, k, opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k, opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2 + cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

In [49]:
def cruza_padres(problema_genetico, padres):
    N = len( padres )
    assert N % 2 == 0
    poblacionCruzada = list()
    for i in range( N // 2 ):
        cruces = problema_genetico.cruza( padres[2 * i], padres[2 * i + 1] )
        poblacionCruzada.append(cruces[0].copy())
        poblacionCruzada.append(cruces[1].copy())
        
    return poblacionCruzada

In [50]:
def muta_individuos(problema_genetico, poblacion, prob):
    N = len( poblacion )
    for x in poblacion:
        problema_genetico.muta(x, prob)
        
    return poblacion

#### Clase del problema de asignación.

Definimos una clase del problema de asignación, que se inicializa con la matriz de tiempos de ejecución y la matriz de interdependencias.

In [93]:
class ProblemaAsignacion(ProblemaGenetico):
    def __init__(self, genes, fun_dec, fun_muta, fun_cruza, fun_fitness, nTrab, mTarea, tiemposMat, depMat):
        self.genes = genes
        self.fun_dec = fun_dec
        self.fun_cruza = fun_cruza
        self.fun_muta = fun_muta
        self.fun_fitness = fun_fitness
        self.longitud_individuos = mTarea
        self.nTrab = nTrab
        self.mTarea = mTarea
        self.tiemposMat = tiemposMat
        self.depMat = depMat
        
    def decodifica(self, genotipo):
        fenotipo = self.fun_dec(self.nTrab, self.mTarea, genotipo, self.tiemposMat, self.depMat)
        
        return fenotipo
    
    def fitness(self, cromosoma):
        valoracion = self.fun_fitness(self.nTrab, self.mTarea, cromosoma, self.tiemposMat, self.depMat)
        
        return valoracion
    
    def cruza(self, cromosoma1, cromosoma2):
        cruce = self.fun_cruza(cromosoma1, cromosoma2)
        
        return cruce

    def muta(self, cromosoma, prob):
        mutacion = self.fun_muta(self.nTrab, self.mTarea, cromosoma, prob)
        
        return mutacion

#### Ejemplo 1.

In [94]:
n1 = 2
m1 = 3
i = 999999
tiemposMat1 = [ [1, 5, i],
                [i, 3, 2] ]
depMat1 = [ [0, 0, 0],
            [0, 0, 0],
            [0, 1, 0] ]
asignacion1 = ProblemaAsignacion(
    list(range(n1)),
    decodeAsig,
    mutacionAsig,
    cruceAsig,
    fitnessAsig,
    n1,
    m1,
    tiemposMat1,
    depMat1
)

((((0, 0), (0, 1)), ((1, 1), (0, 3)), ((1, 2), (3, 5))), 5)

In [131]:
import operator

minDict = dict()
for _ in range(1000):
    result = algoritmo_genetico(asignacion1, 3, min, 20, 12, 0.75, 0.25)
    if result in minDict:
        minDict[result] += 1
    else:
        minDict[result] = 1

minSorted = sorted(minDict.items(), key = operator.itemgetter(1), reverse = True)
for result in minSorted:
    print(result[0], ': ', round( result[1] / 10, 2), ' %' )

((((0, 0), (0, 1)), ((1, 1), (0, 3)), ((1, 2), (3, 5))), 5) :  93.3  %
((((1, 0), (0, 999999)), ((0, 1), (0, 5)), ((1, 2), (999999, 1000001))), 1000001) :  6.7  %


En este ejemplo simple, el algoritmo genético consigue alcanzar la solución óptima la mayoría de las veces.

#### Ejemplo 2.

In [136]:
n2 = 5
m2 = 5
i = 999999

tiemposMat2 = [ [1, 5, i, 4, 6],
                [i, 3, 2, 1, i],
                [7, 1, 8, 8, 9],
                [i, i, 1, 2, 3],
                [i, 2, i, 4, i] ]

depMat2 = [ [0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0],
            [0, 1, 0, 0, 0],
            [0, 0, 1, 0, 0], 
            [0, 1, 0, 1, 0] ]

asignacion2 = ProblemaAsignacion(
    list(range(n2)),
    decodeAsig,
    mutacionAsig,
    cruceAsig,
    fitnessAsig,
    n2,
    m2,
    tiemposMat2,
    depMat2
)

In [143]:
minDict = dict()
for _ in range(1000):
    result = algoritmo_genetico(asignacion2, 3, min, 20, 15, 0.75, 0.25)
    if result[1] in minDict:
        minDict[result[1]] += 1
    else:
        minDict[result[1]] = 1

minSorted = sorted(minDict.items(), key = operator.itemgetter(1), reverse = True)
for result in minSorted:
    print(result[0], ': ', round( result[1] / 10, 2), ' %' )

8 :  30.3  %
7 :  24.5  %
9 :  11.6  %
10 :  10.6  %
13 :  5.9  %
11 :  5.3  %
14 :  4.7  %
12 :  3.7  %
15 :  1.4  %
17 :  0.8  %
16 :  0.7  %
18 :  0.4  %
1000006 :  0.1  %


En este segundo ejemplo, vemos que la mayoría de soluciones se encuentran muy cercanas a una solución que podemos suponer óptima (7), pero no es la la que más se da, superada por (8).

#### Ejemplo 3.

In [138]:
n3 = 5
m3 = 9
i = 999999

tiemposMat3 = [ [1, 5, i, 8, 2, 1, i, 9, 1],
                [8, 7, 3, i, i, 5, 1, 7, 4],
                [i, 2, 5, 7, 8, 6, 4, i, 9],
                [7, 8, 9, 8, 7, 8, 9, 8, 7],
                [i, i, 2, 1, i, 1, i, 2, 1] ]

depMat3 = [ [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [1, 0, 1, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 1, 1, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0],
            [0, 0, 0, 1, 1, 0, 0, 0, 0],
            [0, 1, 0, 1, 0, 0, 1, 0, 0] ]

asignacion3 = ProblemaAsignacion(
    list(range(n3)),
    decodeAsig,
    mutacionAsig,
    cruceAsig,
    fitnessAsig,
    n3,
    m3,
    tiemposMat3,
    depMat3
)

In [139]:
minDict = dict()
for _ in range(100):
    result = algoritmo_genetico(asignacion3, 5, min, 20, 50, 0.75, 0.25)
    if result[1] in minDict:
        minDict[result[1]] += 1
    else:
        minDict[result[1]] = 1

minSorted = sorted(minDict.items(), key = operator.itemgetter(1), reverse = True)
for result in minSorted:
    print(result[0], ': ', round( result[1], 2), ' %' )

14 :  17  %
10 :  11  %
19 :  10  %
18 :  9  %
17 :  9  %
13 :  8  %
15 :  7  %
20 :  7  %
9 :  4  %
11 :  4  %
8 :  3  %
16 :  3  %
12 :  2  %
5 :  2  %
26 :  1  %
25 :  1  %
27 :  1  %
6 :  1  %


In [144]:
optimal = ((), 99999999)
for _ in range(100):
    result = algoritmo_genetico(asignacion2, 5, min, 20, 50, 0.75, 0.25)
    if result[1] < optimal[1]:
        optimal = result

print(optimal)

((((0, 0), (0, 1)), ((2, 1), (1, 2)), ((3, 2), (2, 3)), ((1, 3), (3, 4)), ((3, 4), (4, 7))), 7)


En este último ejemplo, con un mayor número de tareas, el comportamiento del algoritmo se vuelve menos estable, consiguiendo en la mayoría de casos soluciones subóptimas, pero aportando las soluciones más óptimas con menos frecuencia, como (5). Esto no impide, sin embargo, que no podamos hallar esas soluciones más óptimas aplicando el algoritmo genético numerosas veces para mejorar la solución.

Este último ejemplo pone en relieve la complejidad del problema planteado.